In [41]:
# 读取数据 用电影的id来表示电影 便于计算

def load_movielens(path='ml-100k'):
    movies = {}
    for line in open(path + '/u.item', encoding='latin-1'):
        id, title = line.split('|')[0:2]
        movies[id] = id

    prefs = {}
    for line in open(path + '/u.data', encoding='latin-1'):
        user, movieid, rating, ts = line.split('\t')
        prefs.setdefault(user, {})
        prefs[user][movies[movieid]] = float(rating)
    return prefs
prefs= load_movielens()
prefs['1']


{'1': 5.0,
 '10': 3.0,
 '100': 5.0,
 '101': 2.0,
 '102': 2.0,
 '103': 1.0,
 '104': 1.0,
 '105': 2.0,
 '106': 4.0,
 '107': 4.0,
 '108': 5.0,
 '109': 5.0,
 '11': 2.0,
 '110': 1.0,
 '111': 5.0,
 '112': 1.0,
 '113': 5.0,
 '114': 5.0,
 '115': 5.0,
 '116': 3.0,
 '117': 3.0,
 '118': 3.0,
 '119': 5.0,
 '12': 5.0,
 '120': 1.0,
 '121': 4.0,
 '122': 3.0,
 '123': 4.0,
 '124': 5.0,
 '125': 3.0,
 '126': 2.0,
 '127': 5.0,
 '128': 4.0,
 '129': 5.0,
 '13': 5.0,
 '130': 3.0,
 '131': 1.0,
 '132': 4.0,
 '133': 4.0,
 '134': 4.0,
 '135': 4.0,
 '136': 3.0,
 '137': 5.0,
 '138': 1.0,
 '139': 3.0,
 '14': 5.0,
 '140': 1.0,
 '141': 3.0,
 '142': 2.0,
 '143': 1.0,
 '144': 4.0,
 '145': 2.0,
 '146': 4.0,
 '147': 3.0,
 '148': 2.0,
 '149': 2.0,
 '15': 5.0,
 '150': 5.0,
 '151': 4.0,
 '152': 5.0,
 '153': 3.0,
 '154': 5.0,
 '155': 2.0,
 '156': 4.0,
 '157': 4.0,
 '158': 3.0,
 '159': 3.0,
 '16': 5.0,
 '160': 4.0,
 '161': 4.0,
 '162': 4.0,
 '163': 4.0,
 '164': 3.0,
 '165': 5.0,
 '166': 5.0,
 '167': 2.0,
 '168': 5.0,
 '169': 

In [18]:
# 划分数据集 按照7：1 划分数据集
import random
import numpy
def split_data(data, M, k, seed):
    
    test = []
    train = []
    random.seed(seed)
    for user in data:
        if random.randint(0, M) == k:
            test.append([user, data[user]])
        else:
            train.append([user, data[user]])
    return train, test
train,test = split_data(prefs,8,5,1)
def changenp(data):     # 把每个用户的评分放到一个大矩阵中 
    a = []
    b = []
    for i in range(len(data)):
        for j in range(1682):
            a.append(data[i][1].get(str(j+1),0.0))
        b.append(a)
        a = []
    b = numpy.array(b)
    return b

a = changenp(train)
b = changenp(test)
b

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [3., 0., 0., ..., 0., 0., 0.]])

In [2]:
import numpy as np
import math

def cos_sim(x, y):

    numerator = x * y.T  # x 和 y 之间的内积
    a = len(numerator[(numerator>0)])  # 两行之间共同评分的电影个数
    b = x * x.T
    b = len(b[b>0])   # x行评分电影个数
    c = y * y.T
    c = len(c[c>0])   # y行电影评分个数
    denominator = math.sqrt(b*c)
    return (a / denominator)

In [43]:
# 把行与行之间的相似度放到一个 用户个数×用户 个数的矩阵中
def similarity(data):
    """计算矩阵中任意两行之间的相似度
    Args:
    - data: mat, 任意矩阵

    :return: w, mat, 任意两行之间的相似度
    """

    m = np.shape(data)[0]  # 用户的数量
    # 初始化相似矩阵
    w = np.mat(np.zeros((m, m)))

    for i in range(m):
        for j in range(i, m):
            if not j == i:
                # 计算任意两行之间的相似度
                w[i, j] = cos_sim(data[i], data[j])
                w[j, i] = w[i, j]
            else:
                w[i, j] = 0
    return w
w = similarity(a)

In [44]:
import numpy as np

#针对某个用户推荐电影的推荐系数

def user_based_recommend(data, w, user):
    """基于用户相似性为用户 user 推荐物品

    Args:
    - data: mat, 用户物品矩阵
    - w: mat, 用户之间的相似度
    - user: int, 用户编号

    :return: predict, list, 推荐列表
    """
    m, n = np.shape(data)
    interaction = data[user]  # 用户 user 与物品信息

    # 找到用户 user 没有互动过的物品
    not_inter = []
    for i in range(n):
        if interaction[i] == 0:  # 没有互动的物品
            not_inter.append(i)

    # 对没有互动过的物品进行预测
    predict = {}
    for x in not_inter:
        item = np.copy(data[:, x])  # 找到所有用户对商品 x 的互动信息
        for i in range(m):  # 对每一个用户
            if item[i] != 0:
                if x not in predict:
                    predict[x] = w[user, i] * item[i]
                else:
                    predict[x] = predict[x] + w[user, i] + item[i]
    return sorted(predict.items(), key=lambda d: d[1], reverse=True)
predict = user_based_recommend(a,w,1)
predict

[(173, 1656.8367411543252),
 (0, 1641.212336240308),
 (97, 1560.5543196863791),
 (55, 1511.0711358945064),
 (171, 1417.246478070393),
 (78, 1262.4677344153185),
 (172, 1261.2935444225843),
 (203, 1246.2491079398953),
 (317, 1244.6098421201332),
 (209, 1219.3922916331292),
 (167, 1184.2407228488387),
 (63, 1158.9099613582562),
 (21, 1148.2912063160697),
 (68, 1133.7866220232117),
 (150, 1114.075428770227),
 (194, 1109.7854339314122),
 (182, 1101.9859382356315),
 (11, 1094.8076923422316),
 (95, 1089.728221188131),
 (422, 1063.9936933080207),
 (88, 1061.4813466937032),
 (190, 1049.433251120016),
 (175, 1047.8814876650783),
 (215, 1046.818398110886),
 (356, 1037.2861774050439),
 (274, 1017.8839726100426),
 (233, 997.7700462773446),
 (201, 986.4348690155188),
 (27, 976.9786112305362),
 (482, 967.520550449173),
 (741, 959.682120723286),
 (134, 936.1601867884169),
 (237, 924.4326841252579),
 (184, 919.0437333159834),
 (195, 913.0693589277288),
 (131, 901.4348209286469),
 (117, 900.24127763374

In [45]:
def top_k(predict, n):
    """为用户推荐前 n 个物品

    Args:
    - predict: list, 排好序的物品列表
    - k: int, 推荐的物品个数

    :return: top_recom, list, top n 个物品
    """
    top_recom = []
    len_result = len(predict)
    if n >= len_result:
        top_recom = predict
    else:
        for i in range(n):
            top_recom.append(predict[i])
    return top_recom
top = top_k(predict,5)
top

[(173, 1656.8367411543252),
 (0, 1641.212336240308),
 (97, 1560.5543196863791),
 (55, 1511.0711358945064),
 (171, 1417.246478070393)]

In [22]:
# 召回率
def recall(train, test, N,predict):
    hit = 0
    all = 0
    for user in range(len(test)):
        tu = test[user]
        rank = top_k(predict, N)
        for item, pui in rank:
            if item in tu:
                hit += 1
        all += len(tu)
    return hit / (all * 1.0)
recall(a,b,5,predict)

0.0005945303210463733

In [23]:
# 准确度
def precision(train, test, N, predict):
    hit = 0
    all = 0
    for user in range(len(test)):
        tu = test[user]
        rank = top_k(predict, N)
        for item, pui in rank:
            if item in tu:
                hit += 1
        all += N
    return hit / (all * 1.0)
precision(a,b,5,predict)

0.2

In [27]:
# 覆盖率
def coverage(train,N, predict):
    recommend_items = set()
    all_items = set()
    for user in range(len(train)):
        for item in train[user]:
            if item != 0:
                all_items.add(user)
        rank = top_k(predict, N)
        for item, pui in rank:
            recommend_items.add(item)
    return len(recommend_items) / (len(all_items) * 1.0)
coverage(a,5,predict)

0.00591016548463357

In [37]:
# 新颖度
def popularity(train, N, predict):
    item_popularity = dict()
    for user in range(len(train)):
        i = 0
        for item in train[user]:
            i = i+1
            if item != 0:
                item_popularity[str(i)]=item_popularity.get(str(i),0)
    ret = 0
    n = 0
    for user in range(len(train)):
        rank = top_k(predict, N)
        for item, pui in rank:
            ret += math.log(1 + item_popularity.get(item,0.1))
            n += 1
    ret /= n * 1.0
    return ret
popularity(a, 5, predict)

0.09531017980431797